In [1]:
import sys
import os

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.preprocessing.extract import recursive_extract_text

recursive_extract_text('articles/raw/')

File PDF Visual Regression Testing — ‘The PuppetMaster’ Approach _ by Ben Ellerby _ The CRC Tech Blog _ Medium.txt already exists, skipping...
File Are we getting infrastructure all wrong in the Serverless era_ _ by Yan Cui _ theburningmonk.com _ Medium.txt already exists, skipping...
File The one mistake everyone makes when using Kinesis with Lambda _ by Yan Cui _ theburningmonk.com _ Medium.txt already exists, skipping...
File How to secure CI_CD roles without burning production to the ground _ by Yan Cui _ theburningmonk.com _ Feb, 2024 _ Medium.txt already exists, skipping...
File When to use Step Functions vs. doing it all in a Lambda function _ by Yan Cui _ theburningmonk.com _ Mar, 2024 _ Medium.txt already exists, skipping...
File Building a Massively Scalable Serverless Chat Application with AWS AppSync _ by Sarah Hamilton _ Serverless Transformation _ Medium.txt already exists, skipping...
File When to use API Gateway vs. Lambda Function URLs _ by Yan Cui _ theburningmonk.com

In [ ]:
from src.preprocessing.extract import extract_text

pdf_path = 'articles/raw/Building a Massively Scalable Serverless Chat Application with AWS AppSync _ by Sarah Hamilton _ Serverless Transformation _ Medium.pdf'
output_path = 'Building a Massively Scalable Serverless Chat Application with AWS AppSync _ by Sarah Hamilton _ Serverless Transformation _ Medium.txt'

extract_text(pdf_path, output_path)


In [2]:
from src.preprocessing.chunk import recursive_chunk_text
from src.preprocessing.chunk import chunk_text
import os

# delete articles/chunks/chunks.txt
if os.path.exists('articles/chunks/chunks.txt'):
    os.remove('articles/chunks/chunks.txt')
    
recursive_chunk_text('articles/parsed/')
# chunk_text('articles/parsed/Building a Massively Scalable Serverless Chat Application with AWS AppSync _ by Sarah Hamilton _ Serverless Transformation _ Medium.txt')

Processing articles/parsed/How to secure CI_CD roles without burning production to the ground _ by Yan Cui _ theburningmonk.com _ Feb, 2024 _ Medium.txt
Processing articles/parsed/When to use Step Functions vs. doing it all in a Lambda function _ by Yan Cui _ theburningmonk.com _ Mar, 2024 _ Medium.txt
Processing articles/parsed/The one mistake everyone makes when using Kinesis with Lambda _ by Yan Cui _ theburningmonk.com _ Medium.txt
Processing articles/parsed/When to use API Gateway vs. Lambda Function URLs _ by Yan Cui _ theburningmonk.com _ Mar, 2024 _ Medium.txt
Processing articles/parsed/Empowering Enterprises with Serverless Generative AI_  Amazon Bedrock _ GenAI Days.txt
Processing articles/parsed/Serverless webhooks — Designing Lift _ by Frédéric Barthelet _ Serverless Transformation _ Medium.txt
Processing articles/parsed/Announcing the Alpha Release of sls-test-tools — Bringing Simplicity to Serverless Integration Testing _ by Ben Ellerby _ Serverless Transformation _ Mediu

In [1]:
from src.preprocessing.neutralise import create_neutralised_dataset

neutraliser = create_neutralised_dataset('articles/chunks/chunks.txt', 'articles/neutralised/training_data.jsonl')

Neutralising chunk: 1/7
Neutralising chunk: 2/7
Neutralising chunk: 3/7
Neutralising chunk: 4/7
Neutralising chunk: 5/7
Neutralising chunk: 6/7
Neutralising chunk: 7/7


In [4]:
import tiktoken



def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

with open('articles/chunks/chunks.txt', 'r') as file:
    chunks = file.read()

print(num_tokens_from_string(chunks, "cl100k_base"))

31788
